In [1]:
import rdflib

Load ontology and example

In [62]:
g = rdflib.Graph()
g.parse("../ontology/yang-server.ttl", format='turtle')
g.parse("../examples/server-example.ttl", format='turtle')
print(len(g))

472


Load prefixes

In [4]:
prefixes = """
PREFIX ys: <https://w3id.org/yang/server#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX observable: <https://ontology.unifiedcyberontology.org/uco/observable/>
"""

## server-1: A YANG server is accesible through an endpoint composed of URL or IP address and port number

In [7]:
q_res = g.query(prefixes + """
    SELECT ?server ?socketAddressValue
    WHERE {
        ?server rdf:type/rdfs:subClassOf* ys:YangServer ;
            ys:socketAddress ?socketAddress .
        ?socketAddress observable:addressValue ?socketAddressValue
    }
""")

for row in q_res:
    socketValue = row.socketAddressValue.split(":")
    ipAddress = socketValue[0]
    port = socketValue[1]
    print(f"Server: {row.server} IP Address: {ipAddress} Port: {port}")

Server: https://netconf-rml-demo.org/netconf-server-1 IP Address: localhost Port: 830


## server-2: Access to a YANG server requires username and password credentials

In [8]:
q_res = g.query(prefixes + """
    SELECT ?server ?username ?password
    WHERE {
        ?server rdf:type/rdfs:subClassOf* ys:YangServer ;
            ys:serverAccount ?account .
        ?account ys:username ?username ;
            observable:hasFacet ?auth .
        ?auth observable:password ?password
    }
""")

for row in q_res:
    print(f"Server: {row.server} Username: {row.username} Password: {row.password}")

Server: https://netconf-rml-demo.org/netconf-server-1 Username: netconf Password: netconf


## server-3: Access to data of a YANG server can be filtered using XPath

## server-4: NETCONF server is a type of YANG server

In [54]:
q_res = g.query(prefixes + """
    ASK {
        ?server rdf:type ys:NetconfServer
    }
""")

print(f"{q_res.askAnswer}")

True


## server-7: A NETCONF server implements NETCONF capabilities

In [63]:
q_res = g.query(prefixes + """
    SELECT ?server ?nc_capability
    WHERE {
        ?server a ys:NetconfServer ;
            ys:capability ?nc_capability .
        ?nc_capability a ys:NetconfCapability .
    }
""")

for row in q_res:
    print(f"{row.nc_capability}")

https://w3id.org/yang/server#XpathCapability
https://w3id.org/yang/server#YangLibrary1.0Capability


## server-8: A NETCONF capability is uniquely identified by a URN

In [65]:
q_res = g.query(prefixes + """
    SELECT ?nc_capability ?id
    WHERE {
        ?server a ys:NetconfServer ;
            ys:capability ?nc_capability .
        ?nc_capability a ys:NetconfCapability ;
            ys:netconfCapabilityId ?id .
    }
""")

for row in q_res:
    print(f"{row.nc_capability} --> ID: {row.id}")

https://w3id.org/yang/server#XpathCapability --> ID: urn:ietf:params:netconf:capability:xpath:1.0
https://w3id.org/yang/server#YangLibrary1.0Capability --> ID: urn:ietf:params:netconf:capability:yang-library:1.0


## server-9: Host key verification can be enabled or disabled to access a NETCONF server

In [29]:
q_res = g.query(prefixes + """
    SELECT ?server ?host_key
    WHERE {
        ?server a ys:NetconfServer ;
            ys:hostKeyVerification ?host_key
    }
""")

for row in q_res:
    print(f"Server: {row.server} Enabled: {row.host_key}")

Server: https://netconf-rml-demo.org/netconf-server-1 Enabled: false


## server-10: A YANG server stores YANG data in a datastore

In [34]:
q_res = g.query(prefixes + """
    SELECT ?server ?datastore
    WHERE {
        ?datastore ys:server ?server;
    }
""")

for row in q_res:
    print(f"Server: {row.server} Datastore: {row.datastore}")

Server: https://netconf-rml-demo.org/netconf-server-1 Datastore: https://netconf-rml-demo.org/netconf-server-1/datastores/operational
Server: https://netconf-rml-demo.org/netconf-server-1 Datastore: https://netconf-rml-demo.org/netconf-server-1/datastores/running
Server: https://netconf-rml-demo.org/netconf-server-1 Datastore: https://netconf-rml-demo.org/netconf-server-1/datastores/candidate
Server: https://netconf-rml-demo.org/netconf-server-1 Datastore: https://netconf-rml-demo.org/netconf-server-1/datastores/startup


## server-11: A conventional configuration datastore is a type of datatore that holds configuration

In [69]:
q_res = g.query(prefixes + """
    SELECT ?server ?datastore
    WHERE {
        ?datastore ys:server ?server ;
            rdf:type/rdfs:subClassOf* ys:ConventionalDatastore .
    }
""")

for row in q_res:
    print(f"Server: {row.server} Datastore: {row.datastore}")

Server: https://netconf-rml-demo.org/netconf-server-1 Datastore: https://netconf-rml-demo.org/netconf-server-1/datastores/candidate
Server: https://netconf-rml-demo.org/netconf-server-1 Datastore: https://netconf-rml-demo.org/netconf-server-1/datastores/running
Server: https://netconf-rml-demo.org/netconf-server-1 Datastore: https://netconf-rml-demo.org/netconf-server-1/datastores/startup


## server-12: Running, startup, candidate, and intended are types of conventional configuration datastores

In [72]:
q_res = g.query(prefixes + """
    SELECT ?datastore
    WHERE {
        ?datastore rdfs:subClassOf ys:ConventionalDatastore
    }
""")

for row in q_res:
    print(f"{row.datastore}")

https://w3id.org/yang/server#CandidateDatastore
https://w3id.org/yang/server#IntendedDatastore
https://w3id.org/yang/server#RunningDatastore
https://w3id.org/yang/server#StartupDatastore


## server-13: In addition to convention configuration datastores, a YANG server may have dynamic configuration datastores holding configuration obtained dynamically

In [41]:
q_res = g.query(prefixes + """
    ASK {
        ys:DynamicDatastore rdfs:subClassOf ys:Datastore
    }
""")

print(f"{q_res.askAnswer}")

True


## server-14: A YANG server stores operational state data in the operational state datastore

In [73]:
q_res = g.query(prefixes + """
    SELECT ?server ?datastore
    WHERE {
        ?datastore ys:server ?server ;
            rdf:type ys:OperationalDatastore .
    }
""")

for row in q_res:
    print(f"Server: {row.server} Datastore: {row.datastore}")

Server: https://netconf-rml-demo.org/netconf-server-1 Datastore: https://netconf-rml-demo.org/netconf-server-1/datastores/operational


## server-15: A YANG datastore must be specified when accessing data of a YANG server

In [74]:
q_res = g.query(prefixes + """
    SELECT ?server ?datastore
    WHERE {
        ?server rdf:type/rdfs:subClassOf* ys:YangServer ;
            ys:accessDatastore ?datastore
    }
""")

for row in q_res:
    print(f"{row.server} {row.datastore}")

## server-16: A query to a YANG server must specify datastore and filter

In [22]:
q_res = g.query(prefixes + """
    SELECT ?server ?datastore ?filterValue
    WHERE {
        ?query a ys:Query ;
            ys:sourceServer ?server ;
            ys:sourceDatastore ?datastore ;
            ys:filter ?filter ;
        .
        ?subproperty rdfs:subPropertyOf ys:filterValue .
        ?filter ?subproperty ?filterValue .
    }
""")

for row in q_res:
    print(f"{row.server} {row.datastore} {row.filterValue}")

https://example.org/netconf-server-1 https://example.org/netconf-server-1/datastores/operational /yanglib:modules-state/yanglib:module
https://example.org/netconf-server-1 https://example.org/netconf-server-1/datastores/running 
        <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
          <interface>
          </interface>
        </interfaces>
        


## server-17: Xpath is a type of filter

In [23]:
q_res = g.query(prefixes + """
    ASK {
        ys:XPathFilter rdfs:subClassOf ys:Filter
    }
""")

print(f"{q_res.askAnswer}")

True


## server-18: XML subtree is a type of filter

In [24]:
q_res = g.query(prefixes + """
    ASK {
        ys:SubtreeFilter rdfs:subClassOf ys:Filter
    }
""")

print(f"{q_res.askAnswer}")

True
